In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import re
from scipy.optimize import curve_fit

In [20]:
def load_file(filenm):
    """
    Load xvg - file and return a pandas dataframe. 
    """
    data = []
    with open(filenm) as f:
        for line in f:
            #If line starts with @ or #, skip
            if line.startswith(('#', '@')):
                continue
            data.append([float(x) for x in re.split(r'\s+', line.strip())])
    return pd.DataFrame(data, columns=['time', 'value'])

In [3]:
def get_line_from_file(file_path, line_number):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        if 0 <= line_number < len(lines):
            return lines[line_number].strip()
        else:
            return None

In [4]:
def plot_variable_vs_reference(df, ref, name=None):
    """
    Plots reference distribution as normalised histogram, and variable as kde-plot.
    Also shows the value of the variable used in the simulation by a red line.
    """
    sns.histplot(ref.value, stat='density', label='Reference', bins=50, kde=True)
    sns.kdeplot(df.value,label='Variable', color='red', alpha=0.5)
    ymin, ymax = plt.ylim()
    plt.legend()
    plt.title(name)
    plt.show()

In [ ]:
for var in ['angles','dihedrals']:
    for path in sorted(glob.glob(f"../colvars/CG/MOTOR120/{var}_mapped/{var[0]}*.xvg")):
        varnr = path.split('/')[-1][:-4]
        CG = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/CG/MOTOR*/{var}_mapped/{varnr}.xvg")], axis=0, ignore_index=True)
        AA = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/AA/MOTOR*/{var}_mapped/{varnr}.xvg")], axis=0, ignore_index=True)
        varnm = varnr.split('_')[0]
        varnr = int(varnr.split('_')[1])
        varnm = varnm + ' ' + '-'.join(re.split('\s+',get_line_from_file(f'../structure_files/{var}.ndx', varnr*2 + 1)))
        #Print mean, std and min/max of CG and AA
        print(f'{varnm}:')
        print(f'CG: {CG.value.mean():.2f} +- {CG.value.std():.2f} ({CG.value.min():.2f}, {CG.value.max():.2f})')
        print(f'AA: {AA.value.mean():.2f} +- {AA.value.std():.2f} ({AA.value.min():.2f}, {AA.value.max():.2f})')
        #Print number of values in CG larger than 4
        print(f'AA Shape: {AA.shape}')
        print(f'CG Shape: {CG.shape}')
        plot_variable_vs_reference(CG, AA, name=varnm)

In [ ]:
import sys, os
from scipy.stats import gaussian_kde
# --- bring in the styler
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from md_styler import MDStyler
sty = MDStyler().apply()
width=45
# convenience colors & line styles
c_aa = sty.get_color("cyan")      # AA color
c_cg = sty.get_color("box")       # CG color (deep blue)
kw_aa = sty.as_aa()
kw_cg = sty.as_cg()

def kde_mode_center(x):
    """Return the mode (peak) of a 1D distribution."""
    x = np.asarray(x)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return 0.0
    kde = gaussian_kde(x, bw_method="scott")
    xs = np.linspace(x.min(), x.max(), 1024)
    ys = kde(xs)
    return xs[np.argmax(ys)]

def kde_plot(ax, data, color, label, linestyle):
    sns.kdeplot(data, ax=ax, color=color, linestyle=linestyle, linewidth=2.0, label=label, bw_method="scott")

# --- load data
fig, ax = plt.subplots(1, 3, figsize=(11.6, 3.4))  # aspect not critical; x-scale will be identical

# Panel 1: Dihedral 1
df = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/CG/MOTOR*/dihedrals_mapped/dih_3.xvg")], axis=0, ignore_index=True)
ref = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/AA/MOTOR*/dihedrals_mapped/dih_3.xvg")], axis=0, ignore_index=True)
kde_plot(ax[0], -np.abs(ref.value), color=c_aa, label="AA", linestyle=kw_aa["linestyle"])
kde_plot(ax[0], -np.abs(df.value),  color=c_cg, label="CG", linestyle=kw_cg["linestyle"])
peak = kde_mode_center(-np.abs(ref.value))
ax[0].set_xlim(peak - width, peak + width)
ax[0].set_title("Dihedral 1: 5–1–8–13")
ax[0].set_xlabel("Angle (degrees)")
ax[0].set_ylabel("Probability density")

# Panel 2: Dihedral 2
df = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/CG/MOTOR*/dihedrals_mapped/dih_4.xvg")], axis=0, ignore_index=True)
ref = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/AA/MOTOR*/dihedrals_mapped/dih_4.xvg")], axis=0, ignore_index=True)
kde_plot(ax[1], ref.value, color=c_aa, label="AA", linestyle=kw_aa["linestyle"])
kde_plot(ax[1], df.value,  color=c_cg, label="CG", linestyle=kw_cg["linestyle"])
peak = kde_mode_center(ref.value)
ax[1].set_xlim(peak - width-10, peak + width+10)
ax[1].set_title("Dihedral 2: 5–7–8–11")
ax[1].set_xlabel("Angle (degrees)")
ax[1].set_ylabel(None)

# Panel 3: Angle 1
df = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/CG/MOTOR*/angles_mapped/ang_0.xvg")], axis=0, ignore_index=True)
ref = pd.concat([load_file(filenm) for filenm in glob.glob(f"../colvars/AA/MOTOR*/angles_mapped/ang_0.xvg")], axis=0, ignore_index=True)
kde_plot(ax[2], ref.value, color=c_aa, label="AA", linestyle=kw_aa["linestyle"])
kde_plot(ax[2], df.value,  color=c_cg, label="CG", linestyle=kw_cg["linestyle"])
peak = kde_mode_center(ref.value)
ax[2].set_xlim(peak - width, peak + width)
ax[2].set_title("Angle 1: 8-9-10")
ax[2].set_xlabel("Angle (degrees)")
ax[2].set_ylabel(None)

# shared legend
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=2, frameon=False, bbox_to_anchor=(0.52, 0.95))
fig.suptitle("Fibre", fontsize=11, fontweight="bold", y=1.02)
plt.tight_layout()
plt.show()


In [ ]:
sns.histplot(CG.value)
sns.histplot(AA.value)